In [ ]:
cd ../

This demo requires that you have a local posgres SQL database already set up. If you have not done this yet, you can download the PostgreSQL installer here: https://www.postgresql.org/download/. Follow the instructions to get your database environment setup.

Once you have your PostgreSQL environment set up, you can upload the example [car dataset](https://github.com/lux-org/lux-datasets/blob/master/data/car.csv) to your database using the script found [here](https://github.com/thyneb19/lux/blob/Database-Executor/lux/data/upload_car_data.py).

To connect Lux to your PostgreSQL database, you will first need to create a psycopg2 connection. After that you will be able to specify this connection in the Lux config, and connect a Lux DataFrame to a table as shown below.

In [ ]:
import lux
import psycopg2
import pandas as pd

connection = psycopg2.connect("host=localhost dbname=postgres user=postgres password=lux")

sql_df = lux.LuxDataFrame()
lux.config.set_SQL_connection(connection)
sql_df.set_SQL_table("cars")

Once the Lux Dataframe has been connected to a database table, the parameters necessary to run Lux' recommendation system will automatically be populated.

In [ ]:
#you can view the variable datatypes here
sql_df.data_type

Now that the connection between your DataFrame and your database has been established, you can leverage all of Lux's visual recommendation tools. For a more in-depth look at Lux's functions, check out the main repository [here](https://github.com/lux-org/lux).

In [ ]:
#call the Lux DataFrame to view general variable distributions and relationships.
#You will see that the DataFrame contains the columns of your database table, but is otherwise empty.
#Data is processed as much as possible on the database end, and is only brought in locally when needed to create visualizations.
sql_df

In [ ]:
#you can specify intents just the same as the default Lux system
from lux.vis import Clause

#here we specify that we are interested in a graph containing the variables 'milespergal' and 'cylinders'
#we also specify that we want to apply a filter 'horsepower > 150' to this visualization
sql_df.set_intent(["milespergal", 'cylinders', Clause(attribute ="horsepower", filter_op=">", value=150)])
sql_df

You can also use Lux's Vis package to generate visualizations without having to pull in or process data from your database manually. Instead, you can specify visualization channels and create graphs as shown below.

In [ ]:
from lux.vis.Vis import Vis
from lux.vis.Vis import Clause

#Create a new Lux Clause for each variable you want to use in your graph
#Specify how you want to use the variable in the graph via the channel parameter.
#The channel parameter will specify whether or not a variable is used on the x or y axis, or used to color datapoints
x_clause = Clause(attribute = "acceleration", channel = "x")
y_clause = Clause(attribute = "milespergal", channel = "y")
color_clause = Clause(attribute = 'cylinders', channel = "color")

#you can also create filters on your data using Lux Clauses like so
filter_clause = Clause(attribute ="origin", filter_op="=", value='USA')

#to create the graph, create a Lux Vis object with the list of your Clauses as the parameter
new_vis = Vis([x_clause, y_clause, color_clause, filter_clause])

#to fetch the data necessary for the graph, use the refresh_source function.
#the refresh_source function takes in a Lux DataFrame, in this case you can specify the one connected to your database table
new_vis.refresh_source(sql_df)
new_vis